In [11]:
import numpy as np

from sklearn import preprocessing

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances


import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.vgg16 import preprocess_input

model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(86, 128, 3))

In [12]:
def img_retrivial(query):
    query_np = kimage.img_to_array(query)
    query_np = np.expand_dims(query_np, axis=0)
    query_np = preprocess_input(query_np)

    images = []
    lists = [i for i in range(1,252)]
    for index in lists:
        img = kimage.load_img('images/%d.jpg' % (index), target_size=(86, 128))

        img_np = kimage.img_to_array(img)
        img_np = np.expand_dims(img_np, axis=0)
        img_np = preprocess_input(img_np)

        images.append(img_np)

    images_np = np.array(images).squeeze()

    pred_query = model.predict(query_np)
    pred_images = model.predict(images_np)

    pred_query  = np.reshape(pred_query,  (1, -1))
    pred_images = np.reshape(pred_images, (len(lists), -1))

    cos_sim = cosine_similarity(pred_query, pred_images).squeeze()

    def getKey(item):
        return item[0]

    cos_sim = zip(cos_sim.tolist(), lists)
    cos_sim = sorted(cos_sim, key=getKey, reverse=True)
    
    return cos_sim

In [13]:
f = open('result.txt','w+',encoding = 'utf-8')

Q = ['q1.jpg','q2.jpg','q3.jpg','q4.jpg','q5.jpg']
k = 1
for q in Q:
    query = kimage.load_img(q, target_size=(86, 128))
    
    cos_sim = img_retrivial(query)
    
    f.write('THỰC HIỆN TRUY VẤN ẢNH THỨ {z}'.format(z = k))
    f.write('\n')
    for i in range(8):
        f.write(' - Kết quả của ảnh truy vấn giống thứ {t} của các bộ đo -'.format(t = i+1))
        f.write('\n')
        f.write('Bộ đo cô sin: {t}'.format(t = cos_sim[i][0]))
        f.write('\n')
        f.write('Ảnh được truy vấn có số thứ tự: {t}'.format(t = cos_sim[i][1]))
        f.write('\n')
        f.write('===============================================')
        f.write('\n')
    f.write('\n')
    k+=1